In [3]:
import os
from pathlib import Path
import numpy as np
import cv2
from skimage.morphology import skeletonize, disk

In [4]:
def imwrite_unicode(path, img):
    path = str(path)
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True

def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    try:
        path = str(path)
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None

In [5]:
def setup_project_path():
    current = Path.cwd()
    while True:
        if (current / "craft").exists():
            return current
        if current.parent == current:
            raise FileNotFoundError("프로젝트 루트를 찾지 못했습니다. (craft 폴더가 상위 경로에 없음)")
        current = current.parent
project_root = setup_project_path()

In [6]:
def normalize_stroke_width(
    img: np.ndarray,
    target_stroke_px: int,
    binarize_method: str = "otsu",
    invert: bool = True,
    open_kernel: int = 3,
):

    if target_stroke_px <= 0:
        raise ValueError("target_stroke_px는 1 이상의 정수여야 합니다.")

    # grayscale
    if img is None:
        raise ValueError("입력 이미지가 None입니다.")
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    if binarize_method == "adaptive":
        bin_img = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            31, 5
        )
    else:
        _, bin_img = cv2.threshold(
            gray, 0, 255,
            cv2.THRESH_BINARY + cv2.THRESH_OTSU
        )

    if invert:
        bin_img = 255 - bin_img

    if open_kernel and open_kernel >= 3:
        k = np.ones((open_kernel, open_kernel), np.uint8)
        bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    binary_bool = bin_img > 0
    skel_bool = skeletonize(binary_bool)

    radius = max(1, target_stroke_px // 2)
    selem = disk(radius) 
    rendered = cv2.dilate(skel_bool.astype(np.uint8), selem, iterations=1)

    normalized_mask = (rendered > 0).astype(np.uint8) * 255
    return normalized_mask


In [7]:
def process_folder_stroke_normalization(
    image_folder: str | Path,
    output_folder: str | Path,
    target_stroke_px: int,
    read_flags=cv2.IMREAD_COLOR,
    binarize_method: str = "otsu",
    invert: bool = True,
    open_kernel: int = 3,
    recursive: bool = False,
    exts=(".png", ".jpg", ".jpeg", ".bmp", ".webp", ".tif", ".tiff"),
    output_ext: str = ".png",
    suffix: str = "_norm",
):

    image_folder = Path(image_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    if recursive:
        files = [p for p in image_folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]
    else:
        files = [p for p in image_folder.iterdir() if p.is_file() and p.suffix.lower() in exts]

    if not files:
        print(f"[INFO] 처리할 이미지가 없습니다: {image_folder}")
        return

    print(f"[INFO] 입력 폴더: {image_folder}")
    print(f"[INFO] 출력 폴더: {output_folder}")
    print(f"[INFO] 대상 이미지 수: {len(files)}")
    print(f"[INFO] target_stroke_px={target_stroke_px}, binarize_method={binarize_method}, invert={invert}")

    ok_count, fail_count = 0, 0

    for p in files:
        img = imread_unicode(p, flags=read_flags)
        if img is None:
            print(f"[FAIL] read: {p}")
            fail_count += 1
            continue

        try:
            norm_mask = normalize_stroke_width(
                img,
                target_stroke_px=target_stroke_px,
                binarize_method=binarize_method,
                invert=invert,
                open_kernel=open_kernel,
            )
        except Exception as e:
            print(f"[FAIL] process: {p} -> {e}")
            fail_count += 1
            continue

        out_path = output_folder / f"{p.stem}{suffix}{output_ext}"
        if not imwrite_unicode(out_path, norm_mask):
            print(f"[FAIL] write: {out_path}")
            fail_count += 1
        else:
            ok_count += 1

    print(f"[DONE] 성공: {ok_count}, 실패: {fail_count}")


In [45]:
image_folder = project_root  / "images"
output_folder = project_root / "images" / "images_normalized"

TARGET_STROKE_PX = 15      
BIN_METHOD = "otsu"        # "otsu" or "adaptive"
INVERT = True              
OPEN_KERNEL = 3            
RECURSIVE = False          

process_folder_stroke_normalization(
    image_folder=image_folder,
    output_folder=output_folder,
    target_stroke_px=TARGET_STROKE_PX,
    read_flags=cv2.IMREAD_COLOR,  
    binarize_method=BIN_METHOD,
    invert=INVERT,
    open_kernel=OPEN_KERNEL,
    recursive=RECURSIVE,
    output_ext=".png",
    suffix="",
)

[INFO] 입력 폴더: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\images
[INFO] 출력 폴더: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\images\images_normalized
[INFO] 대상 이미지 수: 10
[INFO] target_stroke_px=15, binarize_method=otsu, invert=True
[DONE] 성공: 10, 실패: 0
